In [1]:
import numpy as np
from scipy.optimize import minimize_scalar, minimize


In [2]:
# Initial inventory
x_1 = 160 

# Sale price after the fourth month
r = 5 

# Demand functions
def d_1(p):
    return max(120 - 1.5 * p, 0)

def d_2(p):
    return max(90 - 1.5 * p, 0)

def d_3(p):
    return max(80 - 1.5 * p, 0)

def d_4(p):
    return max(50 - 2 * p, 0)

In [3]:
def total_revenue(p):
    # Calculate the demand for each month
    d1 = d_1(p)
    d2 = d_2(p)
    d3 = d_3(p)
    d4 = d_4(p)

    # Calculate the sold quantity each month
    q1 = min(d1, x_1)
    x_2 = x_1 - q1
    q2 = min(d2, x_2)
    x_3 = x_2 - q2
    q3 = min(d3, x_3)
    x_4 = x_3 - q3
    q4 = min(d4, x_4)

    # Unsold items are sold at r
    q5 = x_1 - q1 - q2 - q3 - q4

    # Total revenue
    revenue = q1*p + q2*p + q3*p + q4*p + q5*r

    # We will minimize the negative revenue
    return -revenue


In [4]:
res = minimize_scalar(total_revenue, bounds=(0, 120/1.5), method='bounded')

print('The optimal price is:', np.round(res.x, 2))
print('The maximum revenue is:', round(-res.fun))


The optimal price is: 34.72
The maximum revenue is: 4775


In [5]:
def total_revenues(prices):
    p1, p2, p3, p4 = prices
    # Calculate the demand for each month
    d1 = d_1(p1)
    d2 = d_2(p2)
    d3 = d_3(p3)
    d4 = d_4(p4)

    # Calculate the sold quantity each month
    q1 = min(d1, x_1)
    x_2 = x_1 - q1
    q2 = min(d2, x_2)
    x_3 = x_2 - q2
    q3 = min(d3, x_3)
    x_4 = x_3 - q3
    q4 = min(d4, x_4)

    # Unsold items are sold at r
    q5 = x_1 - q1 - q2 - q3 - q4

    # Total revenue
    revenue = q1*p1 + q2*p2 + q3*p3 + q4*p4 + q5*r

    # We will minimize the negative revenue
    return -revenue

# Initial guess
p0 = [20, 20, 20, 20]

# Perform the optimization
res2 = minimize(total_revenues, p0, bounds=[(0, 120/1.5)]*4)

print('The optimal prices are:', np.round(res2.x, 2))
print('The maximum revenue is:', round(-res2.fun))



The optimal prices are: [42.5  32.5  29.17 15.  ]
The maximum revenue is: 5120


In [6]:
print('Percent improvement to revenues: ', np.round((res2.fun / res.fun - 1) * 100, 2), 'percent.' )

Percent improvement to revenues:  7.21 percent.


In [7]:
# Optimal price for the first month from the previous steps
p1_optimal = res2.x[0] 

print('Optimal price for the first month from the previous steps: ', np.round(p1_optimal,2))
print('At that price, you estimated to sell ', round(min(d_1(p1_optimal), x_1)), 'units.')

q1_realized = 70
print('Say you ended up selling ', q1_realized, 'units.')

# Let's reoptimize


Optimal price for the first month from the previous steps:  42.5
At that price, you estimated to sell  56 units.
Say you ended up selling  70 units.


In [8]:
# Revenue from the first month
revenue_month_1 = q1_realized * p1_optimal

# Inventory update
x_2 = x_1 - q1_realized

def total_revenues_updated(prices):
    p2, p3, p4 = prices
    # Calculate the demand for each month
    d2 = d_2(p2)
    d3 = d_3(p3)
    d4 = d_4(p4)

    # Calculate the sold quantity each month
    q2 = min(d2, x_2)
    x_3 = x_2 - q2
    q3 = min(d3, x_3)
    x_4 = x_3 - q3
    q4 = min(d4, x_4)

    # Unsold items are sold at r
    q5 = x_2 - q2 - q3 - q4

    # Total period 2 and onwards revenue
    revenue = q2*p2 + q3*p3 + q4*p4 + q5*r

    # We will minimize the negative revenue
    return -revenue

# Initial guess
p0 = [30, 30, 10]

# Perform the optimization
res = minimize(total_revenues_updated, p0, bounds=[(0, 90/1.5), (0, 80/1.5), (0, 50/2)])

# Compute total maximum revenue
total_max_revenue = revenue_month_1 - res.fun

print('The optimal prices are:', np.round(res.x, 2))
print('The maximum revenue is:', round(total_max_revenue))

The optimal prices are: [34.   30.67 16.5 ]
The maximum revenue is: 5624
